In [1]:
import os
import json
import pandas as pd
import traceback

# https://python.langchain.com/docs/integrations/chat/openai
from langchain_anthropic import ChatAnthropic

In [2]:
from dotenv import load_dotenv

load_dotenv()

Python-dotenv could not parse statement starting at line 1


True

In [3]:
ANTHROPIC_APIKEY = os.getenv("ANTHROPIC_APIKEY")

In [4]:
# Initialize ChatopenAI
llm_anth = ChatAnthropic(anthropic_api_key=ANTHROPIC_APIKEY, model_name="claude-3-sonnet-20240229",temperature=0.5)

In [5]:
from langchain_anthropic import ChatAnthropic
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import  SequentialChain
from langchain_community.callbacks import get_openai_callback

In [6]:
# JSON response for the chatbot
RESPONSE_JSON_ANTH = {
    "1": {
        "mcq":"multiple choice question",
        "options":{"a":"choice here",
                   "b":"choice here",
                   "c":"choice here",
                   "d":"choice here"
                   },
        "correct":"correct_answer",
    },
    "2": {
        "mcq":"multiple choice question",
        "options":{"a":"choice here",
                   "b":"choice here",
                   "c":"choice here",
                   "d":"choice here"
                   },
        "correct":"correct_answer",
    },
    "3": {
        "mcq":"multiple choice question",
        "options":{"a":"choice here",
                   "b":"choice here",
                   "c":"choice here",
                   "d":"choice here"
                   },
        "correct":"correct_answer",
    },
    "4": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct_answer"
    },
    "5": {
        "mcq":"multiple choice question",
        "options":{"a":"choice here",
                   "b":"choice here",
                   "c":"choice here",
                   "d":"choice here"
                   },
        "correct":"correct_answer",
    }
}

In [7]:
# Template 1 for the prompt
TEMPLATE_ANTH ="""
text:{text}
You are an expert MCQ maker. Give the above text, it is your job to\
create a quize of {number} multiple choice questions for {subject} in a {tone} tone\
Make sure questions are not repeated and check all question to be confirming to the text as well. \
Make sure to format your resepomse like RESPONSE_JSON_ANTH below and use it as a guide.  \
Ensure to make {number} MCQs
###RESPONSE_JSON_ANTH
{response_json}
"""


In [8]:
# Quiz generation prompt
quiz_generation_prompt_anth = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE_ANTH
)

In [9]:
# Quiz generation chain
quize_chain_anth = LLMChain(llm=llm_anth, prompt=quiz_generation_prompt_anth, output_key="quiz", verbose=True)

In [10]:
# Template 2 for the prompt
TEMPLATE2_ANTH="""
Your are an expert English grammerian. Give a multiple choice quiz for {subject} students. \
    you need to evaluate the complexity of the question and give a complete analysis of the quiz. 
    Only use at max 50 words for the complexity analysis. \
    If the questions are not in par with the student's cognitive and analytical abilities of the student, \
    Update the questions which needs to be changed and change the tone such that it perfectly fits the student's cognitive and analytical abilities. \
    Quiz_MCQs: 
    {quiz}

    check from an expert English writer of the above quiz:
    """

In [11]:
# Question evaluation prompt
quiz_evaluation_prompt_anth = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2_ANTH
)

In [12]:
# Quiz evaluation chain
review_chain_anth = LLMChain(llm=llm_anth, prompt=quiz_evaluation_prompt_anth, output_key="review", verbose=True)

In [13]:
# Sequential chain for quiz generation and evaluation
generate_evaluate_chain_anth = SequentialChain(  
    chains=[quize_chain_anth, review_chain_anth],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True
)

In [14]:
# File path for the Text data
file_path = r"/home/pjlinux/projects/mcqgen/data_anth.txt"

In [15]:
# Read the TEXT data
with open(file_path, "r") as file:
    TEXT_ANTH = file.read()

In [16]:
# Serialize python dictionary to JSON
json.dumps(RESPONSE_JSON_ANTH)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct_answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct_answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct_answer"}, "4": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct_answer"}, "5": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct_answer"}}'

In [17]:
NUMBER = 5
SUBJECT = "demand and supply"
TONE = "simple"

In [21]:
# https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

# How to use the token usage tracking
with get_openai_callback() as cb:
    response_anth=generate_evaluate_chain_anth(
    {
        "text": TEXT_ANTH,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON_ANTH)
    }
    )

/home/pjlinux/projects/mcqgen/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

text:Supply and Demand
A glance at how supply and demand work

Over 1.8 million professionals use CFI to learn accounting, financial analysis, modeling and more. Start with a free account to explore 20+ always-free courses and hundreds of finance templates and cheat sheets.

Start Free
Written by Tim Vipond
What are the Laws of Supply and Demand?
The laws of supply and demand are microeconomic concepts that state that in efficient markets, the quantity supplied of a good and quantity demanded of that good are equal to each other. The price of that good is also determined by the point at which supply and demand are equal to each other. Supply and demand are usually expressed in a line graph format, with Quantity (the independent variable) on the y-axis and Price (the dependent variable) on the x-axis.

Understanding Supply
Generally speaking, the supply of a good and its price are dire

In [25]:
# Print token usage
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost}")

Total Tokens: 0
Prompt Tokens: 0
Completion Tokens: 0
Total Cost (USD): $0.0


In [26]:
response_anth

{'text': 'Supply and Demand\nA glance at how supply and demand work\n\nOver 1.8 million professionals use CFI to learn accounting, financial analysis, modeling and more. Start with a free account to explore 20+ always-free courses and hundreds of finance templates and cheat sheets.\n\nStart Free\nWritten by Tim Vipond\nWhat are the Laws of Supply and Demand?\nThe laws of supply and demand are microeconomic concepts that state that in efficient markets, the quantity supplied of a good and quantity demanded of that good are equal to each other. The price of that good is also determined by the point at which supply and demand are equal to each other. Supply and demand are usually expressed in a line graph format, with Quantity (the independent variable) on the y-axis and Price (the dependent variable) on the x-axis.\n\nUnderstanding Supply\nGenerally speaking, the supply of a good and its price are directly proportional to each other and follow a linear relationship. In other words, as pr

In [33]:
quiz_anth =response_anth.get("quiz")

In [34]:
quiz_anth = json.loads(quiz_anth)

In [35]:
# Create a table for the quiz data
quiz_table_data_anth = []
for key, value in quiz_anth.items():
    mcq = value.get("mcq")
    options = " | ".join(
        [
            f"{option} : {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data_anth.append({"MCQ": mcq, "choices": options, "correct": correct})


In [37]:
quiz_table_data_anth

[{'MCQ': 'What is the relationship between supply and price?',
  'choices': 'a : Supply and price are inversely proportional | b : Supply and price are directly proportional | c : There is no relationship between supply and price | d : The relationship between supply and price is non-linear',
  'correct': 'b'},
 {'MCQ': 'What is the relationship between demand and price?',
  'choices': 'a : Demand and price are directly proportional | b : Demand and price are inversely proportional | c : There is no relationship between demand and price | d : The relationship between demand and price is non-linear',
  'correct': 'b'},
 {'MCQ': 'What is the point of equilibrium in the supply and demand model?',
  'choices': 'a : The point where supply and demand lines intersect | b : The point where supply and demand lines are parallel | c : The point where supply and demand lines diverge | d : The point where supply and demand lines are perpendicular',
  'correct': 'a'},
 {'MCQ': 'Which of the followin

In [38]:
quiz = pd.DataFrame(quiz_table_data_anth)

In [39]:
quiz.to_csv("quiz.csv", index=False)